Base에서 실행

# Textcuboid


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## 1) Dataload 20newsgroups  data

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
df_newsgroup = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/20news_dataset_clear/20newsgroup_preprocessed.csv', sep=';', usecols=['target', 'text_cleaned'])
df_newsgroup.rename(columns={'text_cleaned' : 'text'}, inplace=True)

In [3]:
le = LabelEncoder()
le.fit(df_newsgroup['target'].unique())

LabelEncoder()

In [4]:
df_newsgroup['target'] = le.transform(df_newsgroup['target'])

In [5]:
X = df_newsgroup['text'].astype(str)
# y = tf.keras.utils.to_categorical(df_newsgroup['target'], num_classes=df_newsgroup['target'].nunique())
y=list(df_newsgroup['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df_newsgroup['target'])

In [6]:
# 방법2) 케라스 내장함수 활용
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(y_train)
one_hot_test_labels = to_categorical(y_test)

## 2) textcuboid 분류

In [7]:
textcuboid=np.load('./1-Channel textcuboid_20ng(elmo).npy')

In [8]:
textcuboid_test=np.load('./1-Channel textcuboid_test_20ng(elmo).npy')

In [9]:
import keras
from tensorflow.keras import layers

In [10]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
#     keras.callbacks.EarlyStopping(monitor="val_loss", patience=8, verbose=1),
]

In [11]:
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model


input_layer = Input(shape=(952, 256))
conv1 = Conv1D(256, 1, padding='valid', activation='relu')(input_layer)
pooling = GlobalMaxPooling1D()(conv1)

x = Dense(256, activation='relu')(pooling)
x = Dropout(0.5)(x)
output_layer = Dense(20, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)



In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 952, 256)]        0         
                                                                 
 conv1d (Conv1D)             (None, 952, 256)          65792     
                                                                 
 global_max_pooling1d (Globa  (None, 256)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1024)              263168    
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 20)                20500     
                                                             

In [14]:
x_train=textcuboid[1000:]
x_val=textcuboid[:1000]
y_train=one_hot_train_labels[1000:]
y_val=one_hot_train_labels[:1000]

In [16]:
history = model.fit(x_train, y_train,callbacks=callbacks, epochs=30,batch_size=256,validation_data=(x_val, y_val))

Epoch 1/30
55/55 [==============================] - 6s 87ms/step - loss: 1.9141 - accuracy: 0.4433 - val_loss: 0.9496 - val_accuracy: 0.7010 - lr: 0.0010
Epoch 2/30
55/55 [==============================] - 4s 72ms/step - loss: 0.8615 - accuracy: 0.7182 - val_loss: 0.7576 - val_accuracy: 0.7560 - lr: 0.0010
Epoch 3/30
55/55 [==============================] - 4s 71ms/step - loss: 0.6928 - accuracy: 0.7731 - val_loss: 0.6861 - val_accuracy: 0.7750 - lr: 0.0010
Epoch 4/30
55/55 [==============================] - 4s 73ms/step - loss: 0.5917 - accuracy: 0.8052 - val_loss: 0.6703 - val_accuracy: 0.7870 - lr: 0.0010
Epoch 5/30
55/55 [==============================] - 4s 72ms/step - loss: 0.5220 - accuracy: 0.8265 - val_loss: 0.6161 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 6/30
55/55 [==============================] - 4s 71ms/step - loss: 0.4568 - accuracy: 0.8530 - val_loss: 0.5814 - val_accuracy: 0.8150 - lr: 0.0010
Epoch 7/30
55/55 [==============================] - 4s 72ms/step - loss: 0.3

In [17]:
model.evaluate(textcuboid_test,one_hot_test_labels)

118/118 [==============================] - 1s 9ms/step - loss: 0.5814 - accuracy: 0.8367


[0.5814027190208435, 0.8366967439651489]

In [15]:
from keras.models import load_model
model = load_model('best_model.h5')
model.evaluate(textcuboid_test,one_hot_test_labels)

118/118 [==============================] - 2s 8ms/step - loss: 0.5476 - accuracy: 0.8372


[0.547626793384552, 0.8372278213500977]